In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastai.callbacks.tracker import *

torch.backends.cudnn.benchmark = True

import time

In [ ]:
import fastai;fastai.__version__ #safety check

In [ ]:
import torch; torch.__version__ #safety check

In [ ]:
from ranger import Ranger

In [ ]:
from mxresnet import *

In [ ]:
path = untar_data(URLs.IMAGEWOOF_160); path

In [ ]:
tfms = ([

        flip_lr(p=0.5)#,
        #brightness(change=(0.4,0.6)),
        #contrast(scale=(0.7,1.3)),
        #cutout(n_holes=(2,40),length=(5,30),p=.25)

    ], [])

In [1]:
bs=64
size=128

In [ ]:
data = (ImageList.from_folder(path)
        .split_by_folder(valid='val')
        .label_from_folder()
        .transform(tfms=tfms,size=size) 
        .databunch(bs=bs, num_workers=8)  #windows 10 users - num_workers may need to be set to 1 or 0 (if you get pickle fork error)
        .presize(size, scale=(0.35, 1))
        .normalize(imagenet_stats))

In [ ]:
optar = partial(Ranger)

In [ ]:
learn = Learner(data, mxresnet50(), metrics=[accuracy], wd=1e-3,
                opt_func=optar,
                 bn_wd=False, true_wd=True,
                loss_func = LabelSmoothingCrossEntropy())

In [ ]:
#learn.callback_fns += [
#        partial(ShowGraph),
        #partial(SaveModelCallback, name='model-novotest-1')
#    ]

In [ ]:
n = len(learn.data.train_dl)
anneal_start = int(n*5*0.7)

In [ ]:
lr = 1e-2

In [ ]:
res = []
num_epoch=5
for x in range(5):
  learn = Learner(data, mxresnet50(c_out=10), wd=1e-2, opt_func=optar,
                 metrics=[accuracy],
                 bn_wd=False, true_wd=True,
                 loss_func=LabelSmoothingCrossEntropy())
  #n = len(learn.data.train_dl)
  #anneal_start = int(n*5*0.7)
  phase0 = TrainingPhase(anneal_start).schedule_hp('lr', lr)
  phase1 = TrainingPhase(n*5 - anneal_start).schedule_hp('lr', lr, anneal=annealing_cos)
  phases = [phase0, phase1]
  sched = GeneralScheduler(learn, phases)
  learn.callbacks.append(sched)
  learn.fit(num_epoch)
  
  loss, acc = learn.validate()
  res.append(acc.numpy())


In [ ]:
#results:
np.mean(res)
print("------")
res
print("std dev: ",np.std(res))